## Problem statement: 
Connecting users with relevant content gets challenging as the size of data increases.
This code provides two search possibilities through BM25 and Universal Sentence Encoder.
BM25 is the algorithm that is behind solutions such as SOLR. In a nutshell, tt is more of a keyword driven search. There is a nice implementation of BM25 in Python, courtesy of https://github.com/dorianbrown/rank_bm25

Universal Sentence Encoder provides more of semantic understanding of a given text and allows for searches that may be semantically related.

We will perform searches as part of keyword search, and also search for document simility.


## Data
Data is from the Enron email dataset. We will use it to demonstrate capability to search through a given text and get relavant search results.
Any data can be used, with a caveat that Universal Sentence Encoder embeddings 'delute' as the size of a given text grows. 

The original dataset can be found at https://www.cs.cmu.edu/~enron/
This code will use Kaggle's CSV version, which has done a great job of consolidating all the emails into one file. This CSV can be found at https://www.kaggle.com/wcukierski/enron-email-dataset?select=emails.csv

In [ ]:
#import libraries we may need
import pandas as pd
import os, sys, email
from sklearn.utils import shuffle
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
from nltk import sent_tokenize
import nltk
nltk.download('punkt')
from tqdm import tqdm
tqdm.pandas()
from sklearn.metrics.pairwise import cosine_similarity

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!gdown --id 1gY0cz4O7YwiQOMPUMbTw24kC_SCU3iZl --output enron_emails.zip

Downloading...
From: https://drive.google.com/uc?id=1gY0cz4O7YwiQOMPUMbTw24kC_SCU3iZl
To: /content/enron_emails.zip
375MB [00:03, 114MB/s]


In [ ]:
#unzip the data
!unzip enron_emails.zip

Archive:  enron_emails.zip
replace emails.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: emails.csv              


In [ ]:
emails_df = pd.read_csv('emails.csv')

In [ ]:
#due to resource constraints, look at only a sample
#use random_state to ensure the reproducibility of the examples.
#emails_df = shuffle(emails_df).copy()
emails_df = emails_df.sample(n=100000, random_state=1).copy()

In [ ]:
## Helper functions - thanks to Zichen Wang for this code
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [ ]:
# Parse the emails into a list email objects
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)
# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages

emails_df.head()

file  \
186822  jones-t/all_documents/634.   
308790  mann-k/all_documents/5690.   
82383         dasovich-j/sent/423.   
227299          kaminski-v/var/63.   
301824     mann-k/_sent_mail/3208.   

                                           Message-ID  \
186822  <17820178.1075846925335.JavaMail.evans@thyme>   
308790  <29110382.1075845717882.JavaMail.evans@thyme>   
82383    <6812040.1075843194135.JavaMail.evans@thyme>   
227299  <21547648.1075856642126.JavaMail.evans@thyme>   
301824  <12684200.1075846107179.JavaMail.evans@thyme>   

                                         Date                           From  \
186822   Tue, 4 Jan 2000 08:20:00 -0800 (PST)         (tana.jones@enron.com)   
308790  Tue, 15 May 2001 11:03:00 -0700 (PDT)           (kay.mann@enron.com)   
82383   Thu, 28 Sep 2000 08:59:00 -0700 (PDT)      (jeff.dasovich@enron.com)   
227299   Mon, 9 Oct 2000 01:23:00 -0700 (PDT)  (tanya.tamarchenko@enron.com)   
301824  Fri, 13 Oct 2000 01:50:00 -0700 (PDT)           (kay.mann@enron.com)   

                                                  To  \
186822                    (alicia.goodrow@enron.com)   
308790                      (sheila.tweed@enron.com)   
82383                  (christine.piesco@oracle.com)   
227299                    (vince.kaminski@enron.com)   
301824  (lisa.bills@enron.com, ben.jacoby@enron.com)   

                                        Subject Mime-Version  \
186822                               Re: Dinner          1.0   
308790                      Re: Override letter          1.0   
82383                                     Teams          1.0   
227299                   Re: FYI: UK Var issues          1.0   
301824  Change Order #5--Pleasanton Transformer          1.0   

                        Content-Type Content-Transfer-Encoding  \
186822  text/plain; charset=us-ascii                      7bit   
308790  text/plain; charset=us-ascii                      7bit   
82383   text/plain; charset=us-ascii                      7bit   
227299  text/plain; charset=us-ascii                      7bit   
301824  text/plain; charset=us-ascii                      7bit   

                   X-From                         X-To X-cc X-bcc  \
186822         Tana Jones               Alicia Goodrow              
308790           Kay Mann                 Sheila Tweed              
82383       Jeff Dasovich  Christine.Piesco@oracle.com              
227299  Tanya Tamarchenko             Vince J Kaminski              
301824           Kay Mann       Lisa Bills, Ben Jacoby              

                                                X-Folder    X-Origin  \
186822  \Tanya_Jones_Dec2000\Notes Folders\All documents     JONES-T   
308790  \Kay_Mann_June2001_1\Notes Folders\All documents      MANN-K   
82383          \Jeff_Dasovich_Dec2000\Notes Folders\Sent  DASOVICH-J   
227299     \Vincent_Kaminski_Jun2001_5\Notes Folders\Var  Kaminski-V   
301824     \Kay_Mann_June2001_4\Notes Folders\'sent mail      MANN-K   

          X-FileName  \
186822    tjones.nsf   
308790     kmann.nsf   
82383   jdasovic.nsf   
227299   vkamins.nsf   
301824     kmann.nsf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [ ]:
#check our work so far
print(emails_df.shape)
print(emails_df.describe())

(100000, 18)
                                     file  \
count                              100000   
unique                             100000   
top     stokley-c/chris_stokley/sent/360.   
freq                                    1   

                                           Message-ID  \
count                                          100000   
unique                                         100000   
top     <10635759.1075861580948.JavaMail.evans@thyme>   
freq                                                1   

                                         Date                  From  \
count                                  100000                100000   
unique                                  76699                 10153   
top     Wed, 27 Jun 2001 16:02:00 -0700 (PDT)  (kay.mann@enron.com)   
freq                                      225                  3264   

                            To Subject Mime-Version  \
count                    95699  100000        99992   
unique   

##Universal Sentence Encoder
Paper describing the model: https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/46808.pdf

In [ ]:
#used for display only
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
#helper function to get the USE embeddings
def embed(input):
    return model(input)

def embed_keywords(input):
    return model([input])

In [ ]:
#let's do few experiments.Since USE tends to 'water down' the embeddings as text gets longer, we will embed each sentence individually. 
#But, let's also see what happens if we do the embeddings on the entire text
emails_df['processed_sentence'] = emails_df['content'].apply(sent_tokenize)
print("Running embeddings for each sentence within a given text....")
emails_df['sentence_embeddings'] = emails_df['processed_sentence'].progress_map(embed)
print("Running embeddings for an entire text....")
emails_df['whole_text_embeddings'] = emails_df['content'].progress_map(embed_keywords)

  0%|          | 0/100000 [00:00<?, ?it/s]

Running embeddings for each sentence within a given text....


  0%|          | 14/100000 [00:00<12:05, 137.78it/s]

Running embeddings for an entire text....


100%|██████████| 100000/100000 [07:57<00:00, 209.32it/s]


In [ ]:
#take a look at one of the document embeddings
print(emails_df['processed_sentence'].iloc[0])
print("Shape of the 1st document embeddings vector: ", emails_df['sentence_embeddings'].iloc[0].shape)
print("Embeddings for each sentence of a text: ", emails_df['sentence_embeddings'].iloc[0])
print("Embeddings for the whole text: ", emails_df['whole_text_embeddings'].iloc[0])

["It would be nice if you could be at my dinner, since I probably won't know \nanyone else.", "Anytime you want to go to lunch to check on the house status, \nI'd be glad to go..."]
Shape of the 1st document embeddings vector:  (2, 512)
Embeddings for each sentence of a text:  tf.Tensor(
[[-2.8037254e-02 -1.0244286e-02  2.9761788e-02 ...  8.4245134e-05
   1.2398518e-02  3.3947281e-03]
 [ 7.2494321e-02 -4.8069630e-02 -3.7942678e-02 ...  1.5248727e-02
   3.7127510e-02  3.7425518e-02]], shape=(2, 512), dtype=float32)
Embeddings for the whole text:  tf.Tensor(
[[ 0.02080704 -0.07532615 -0.03094116 -0.04379633 -0.01084704 -0.00278797
  -0.04071044  0.01106039  0.05870884 -0.08559489 -0.02174503 -0.05142517
   0.04234329 -0.01064559 -0.11033544 -0.03897433 -0.06515872  0.08520376
  -0.01921097  0.00274822  0.03649516 -0.02962157  0.07895856 -0.02449481
   0.00526776  0.01947003  0.00713179 -0.02251233  0.02954272  0.03544835
   0.02535996  0.05734557  0.05706366  0.00299956  0.01652213  0.10

In [ ]:
#need to optimize - e.g. use batch score comparisons, look at https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder

def find_similarities_sentence_level (search_query, search_target=emails_df):
  #get similarity between search query and each document
  doc_num = 0
  similarity_scores = np.zeros ((len(search_target['sentence_embeddings'])))
  for document in search_target['sentence_embeddings']:
    sent_num = 0
    max_similarity = -1
    for sentence in document:
      cosine_sim = cosine_similarity(np.array(search_query).reshape(1,-1), np.array(sentence).reshape(1,-1))
      if (cosine_sim > max_similarity):
        max_similarity = cosine_sim
      sent_num += 1
    similarity_scores[doc_num] = max_similarity
    doc_num += 1
  return similarity_scores

In [ ]:
def find_similarities_whole_text_level (search_query, search_target=emails_df):
  #get similarity between search query and each document
  doc_num = 0
  similarity_scores = np.zeros ((len(search_target['whole_text_embeddings'])))
  for document in search_target['whole_text_embeddings']:
    similarity_scores[doc_num] = cosine_similarity(np.array(search_query).reshape(1,-1), np.array(document).reshape(1,-1))
    doc_num += 1
  return similarity_scores

In [ ]:
def get_top_n(scores, n=5):
  top_n = np.argsort(scores)[::-1][:n]
  return [i for i in top_n]

In [ ]:
def search_engine(query_data, encoding_level = find_similarities_sentence_level):
  query_data.columns = ['query_text']
  query_data_np = np.array(query_data)
  results = []
  
  for line in query_data_np:
    query = line[0]
    query_embeddings = pd.Series(query).apply(embed_keywords)
    
    #get the similarity scores
    document_similarity_scores = query_embeddings.apply(encoding_level)
    scores = document_similarity_scores[0]
    #look at only the searches with higher relevance scores
    top_n = get_top_n(scores, n=10)
    #result_df = emails_all_df[['Message-ID', 'From', 'To', 'Subject', 'content']].iloc[top_n]
    result_df = emails_df[['content']].iloc[top_n]
    
    if len(result_df) > 0:
      relevance_score = scores[top_n]
      result_df['query'] = query
      result_df['Relevance Score'] = relevance_score
      results.append(result_df.values.tolist())

  flat = [x for sublist in results for x in sublist]
  #column_names = ['Message-ID', 'From', 'To', 'Subject', 'content', 'query', 'Relevance Score']
  column_names = ['content', 'query', 'Relevance Score']
  query_matches = pd.DataFrame(data = flat, columns = column_names)

  return query_matches

In [ ]:
data = {'query_text': ['accounting practices']}
sample_query_df = pd.DataFrame(data=data, columns=['query_text'])
sample_query_df

,query_text
0,accounting practices


In [ ]:
res = search_engine(sample_query_df, encoding_level = find_similarities_sentence_level)

In [ ]:
display(res)

,content,query,Relevance Score
0,"Gareth: Per your voice mail. Marcus thought this was closing next week. I \nwill be out on Monday and Tuesday and will have to find a backup. Let me \nknow if you have further info. Thanks. Sara\n----- Forwarded by Sara Shackleton/HOU/ECT on 11/14/2000 08:37 AM -----\n\n\tTreasa Kirby\n\t11/14/2000 08:15 AM\n\t\t \n\t\t To: Sara Shackleton/HOU/ECT@ECT\n\t\t cc: Brenda L Funk/HOU/ECT@ECT, Marcus Von Bock Und Polach/LON/ECT@ECT\n\t\t Subject: TD crude prepay\n\nSara,\nIn response to your questions below:\n\n1. Rod Nelson will be handling credit as the deal is now being executed by \nENA.\n\n2. Tax is being handled by Steve Jacobson in the London office ( he is our US \ntax person )\n\n3. Accounting is Matt Landy\n\n4. The confirmations are being handled by Kim Theriot and John Wilson is the \ntrader who will be signing on behalf of ENA. We have a call with them today \nat 3.00pm UK time so if there is any change I will let you know.\n\n5. We are still waiting for the term sheet / confirms from TD and are \nexpecting them this afternoon. I will forward them to you asap.\n\n6. TD have proposed using the 98 ISDA which I believe is already in place \nwith ENA and TD Texas.\n\n7. This is just one transaction with 3 individual wti swaps, two of which are \nwith Enron, Enron/TD , TD/Morgan Stanley, Morgan Stanley/ Enron\n\nI will send you the transaction diagram from TD and the ISDA agreement.\n\nIf there is anything else you need please feel free to call me on 0207 783 \n5404\n\nRegards\n\n\n\n---------------------- Forwarded by Treasa Kirby/LON/ECT on 14/11/2000 13:35 \n---------------------------\n\n\nMarcus Von Bock Und Polach\n14/11/2000 11:36\nTo: Treasa Kirby/LON/ECT@ECT\ncc: \n\nSubject: TD crude prepay\n\nas discussed\n---------------------- Forwarded by Marcus Von Bock Und Polach/LON/ECT on \n14/11/2000 11:40 ---------------------------\nFrom: Sara Shackleton on 13/11/2000 11:54 CST\nTo: Marcus Von Bock Und Polach/LON/ECT@ECT\ncc: Gareth Bahlmann/HOU/ECT@ECT \n\nSubject: TD crude prepay\n\nMarcus: Can you please let me know what you need.\nWho is handling credit? tax? accounting?\nWho is booking the confirmations? Have the confirms been drafted or \nnegotiated? Any changes? How many transactions?\n\nSara\n\n\n\n",accounting practices,0.793620
1,"----- Forwarded by Sara Shackleton/HOU/ECT on 12/11/2000 09:46 AM -----\n\n\tTreasa Kirby\n\t11/14/2000 08:15 AM\n\t\t \n\t\t To: Sara Shackleton/HOU/ECT@ECT\n\t\t cc: Brenda L Funk/HOU/ECT@ECT, Marcus Von Bock Und Polach/LON/ECT@ECT\n\t\t Subject: TD crude prepay\n\nSara,\nIn response to your questions below:\n\n1. Rod Nelson will be handling credit as the deal is now being executed by \nENA.\n\n2. Tax is being handled by Steve Jacobson in the London office ( he is our US \ntax person )\n\n3. Accounting is Matt Landy\n\n4. The confirmations are being handled by Kim Theriot and John Wilson is the \ntrader who will be signing on behalf of ENA. We have a call with them today \nat 3.00pm UK time so if there is any change I will let you know.\n\n5. We are still waiting for the term sheet / confirms from TD and are \nexpecting them this afternoon. I will forward them to you asap.\n\n6. TD have proposed using the 98 ISDA which I believe is already in place \nwith ENA and TD Texas.\n\n7. This is just one transaction with 3 individual wti swaps, two of which are \nwith Enron, Enron/TD , TD/Morgan Stanley, Morgan Stanley/ Enron\n\nI will send you the transaction diagram from TD and the ISDA agreement.\n\nIf there is anything else you need please feel free to call me on 0207 783 \n5404\n\nRegards\n\n\n\n---------------------- Forwarded by Treasa Kirby/LON/ECT on 14/11/2000 13:35 \n---------------------------\n\n\nMarcus Von Bock Und Polach\n14/11/2000 11:36\nTo: Treasa Kirby/LON/ECT@ECT\ncc: \n\nSubject: TD crude prepay\n\nas discussed\n---------------------- Forwarded by Marcus Von Bock Und Polach/LON/ECT on \n14/11/2000 11:40 ---------------------------\nFrom

In [ ]:
#now, let's see what results do we get if we do the search based on the whole text embeddings
res2 = search_engine(sample_query_df, encoding_level = find_similarities_whole_text_level)

In [ ]:
display(res2)

,content,query,Relevance Score
0,Accounting Magic. Am I good or what?,accounting practices,0.461578
1,\nELCON commercial practices group[,accounting practices,0.419706
2,"Here is the point person for Devon Accounting questions/information:\n\nGary Wade\nDevon Energy Corporation\nRevenue Accounting Supervisor\ntel 405-552-4721\nwadeg@dvn.com\n20 North Broadway, Suite 1500\nOklahoma City, OK 73102-8260\nFax 405-552-4550",accounting practices,0.412697
3,"Shirley,\n\n2 expense checks:\n\n\n$8,937.30\n$4,971.30\n\n\nVince\n",accounting practices,0.388641
4,"\tKarl,\n\tSorry for the delay in responding. I have spoken to Jim Saunders on the Accounting allocation. He is sending 3% of his time for the accounting information flow to and from Co. 1195 as well as the investment accounting, and time spent on the reorganization accounting matters. The remaining dollars of the accounting allocation are from Rod Hayslett for the time he is spending on Co. 1195 issues. I hope this gives you the information you need.\n\tMary\n\n -----Original Message-----\nFrom: \tGeaccone, Tracy \nSent:\tFriday, October 19, 2001 9:16 AM\nTo:\tBotello, Mary\nSubject:\tFW: Allocations\n\n\n\n -----Original Message-----\nFrom: \tJackson, Karl \nSent:\tThursday, October 18, 2001 6:16 PM\nTo:\tGeaccone, Tracy\nSubject:\tAllocations\n\nI'm getting back to something, I left you a message last week. When Barney presented the ETS budget for the pipeline operations, he included amounts allocated from ETS for IT ($170k) and Accounting ($59k). You left me a phone mail message regarding the IT charge and suggested that I contact Caroline Barnes (which I did today). Both of these appear to be new charges for 2002 and we'd like to get information on what they relate to. Can you shed some light on the Accounting charge?\n\nThanks\nKarl",accounting practices,0.385297
5,"Tracy - \nAs discussed.\nSteve\n\n\n\n \n\nRob Brown\nManager, Enron Corp.\nFinancial Accounting & Reporting\nOff. 713.853.9702\nCell 713.303.4497\nrob.brown@enron.com",accounting practices,0.381349
6,"Good afternoon Sandeep:\n\nI sent Krishna's expenses for his recent trip to our accounting dept. this \nmorning and expensed them to the Research Group's Co and RC#. \nHowever, we need to be reimbursed for these expenses and I would like \nto know the best way to do this. Our accounting dept. thought we should \njust send an invoice with the back up material to Dabhol Power Co. asking \nthem to cut a check made payable to Enron Corp. and then it would be \ncredited to our Co# and RC#.\n\nPlease let me know if this is acceptable, and if so, who the invoice should\nbe made out to and where I should send it.\n\nI appreciate your assistance in this matter.\n\nHave a great day!\n\nShirley Crenshaw\n713-853-5290\n",accounting practices,0.375931
7,"Good afternoon Sandeep:\n\nI sent Krishna's expenses for his recent trip to our accounting dept. this \nmorning and expensed them to the Research Group's Co and RC#. \nHowever, we need to be reimbursed for these expenses and I would like \nto know the best way to do this. Our accounting dept. thought we should \njust send an invoice with the back up material to Dabhol Power Co. asking \nthem to cut a check made payable to Enron Corp. and then it would be \ncredited to our Co# and RC#.\n\nPlease let me know if this is acceptable, and if so, who the invoice should\nbe made out to and where I should send it.\n\nI appreciate your assistance in this matter.\n\nHave a great day!\n\nShirley Crenshaw\n713-853-5290\n",accounting practices,0.375931
8,"Greg,\n\nI work in the accounting group for TechQuest Capital and was wondering if\nyou had a contact for me in your accounting department. We haven't received\npayment for our invoice and I wanted to inquiry about its status.\n\nThank you for your help.\n\nDebi Estey",accounting practices,0.375235
9,"Please get together and make sure that the deferred taxes are moved to the correct entities, reflecting where the reserves are.\n\nPatty\n\

##Results
As expected, the search results based on the embeddings on sentence level tend to be longer texts, the search results based on the embeddings on whole text tend to be shorter. 
In both cases we seem to be getting some relevant results. One of the interesting search results is this one: "Shirley,\n\n2 expense checks:\n\n\n$8,937.30\n$4,971.30\n\n\nVince\n"
Even though there was no mention of 'accounting' per se - that's the boost given by the Universal Sentence Encoder

In [ ]:
#Let's try another search
data = {'query_text': ['upcoming audit']}
sample_query_df = pd.DataFrame(data=data, columns=['query_text'])
sample_query_df

,query_text
0,upcoming audit


In [ ]:
display(search_engine(sample_query_df, encoding_level = find_similarities_sentence_level))

content  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [ ]:
display(search_engine(sample_query_df, encoding_level = find_similarities_whole_text_level))

,content,query,Relevance Score
0,see!,upcoming audit,0.410720
1,"FYI...........\n---------------------- Forwarded by Jeffrey C Gossett/HOU/ECT on 05/07/2001 \n07:49 AM ---------------------------\nFrom: Mechelle Atwood/ENRON@enronXgate on 05/04/2001 03:24 PM\nTo: Robert L Hall/ENRON@enronXgate, Jeffrey C Gossett/HOU/ECT@ECT, Bryce \nBaxter/HOU/ECT@ECT\ncc: Wes Colwell/ENRON@enronXgate, ""john.j.boudreaux@andersen.com"" \n<'john.j.boudreaux@andersen.com'>@SMTP@enronXgate, \n""kate.e.agnew@us.andersen.com"" \n<'kate.e.agnew@us.andersen.com'>@SMTP@enronXgate, Nicole \nMendez/ENRON@enronXgate \nSubject: Gas Trading Audit Notification\n\n\nThe Gas Trading Audit is scheduled to begin May 11, 2001 and is expected to \ntake approximately two to three months to complete. The audit will be \nexecuted by Arthur Andersen and led by Kate Agnew.\n\nWe will schedule an opening meeting with you the week of May 7, 2001 to \ndiscuss audit scope and any concerns or suggestions you may have.\n\nThe objective of the audit will be to determine whether procedures and \ncontrols exercised over activities are adequate and operating effectively. \n\nResults of the audit will be discussed with you and other appropriate \npersonnel both during the audit and at the audit closing meeting. \n\nWe will endeavor to conduct this audit in a manner that will minimize \ninterruption of your normal business activities and we look forward to \nworking with you and your personnel. Please call me at x5-4554 if have any \nquestions. \n\n\nMechelle Atwood\nDirector, Enron Assurance Services\nEnron Wholesale Services\n713.345.4554; Location: EB2383\nMechelle.Atwood@enron.com\n\n",upcoming audit,0.405648
2,"FYI...........\n---------------------- Forwarded by Jeffrey C Gossett/HOU/ECT on 05/07/2001 \n07:49 AM ---------------------------\nFrom: Mechelle Atwood/ENRON@enronXgate on 05/04/2001 03:24 PM\nTo: Robert L Hall/ENRON@enronXgate, Jeffrey C Gossett/HOU/ECT@ECT, Bryce \nBaxter/HOU/ECT@ECT\ncc: Wes Colwell/ENRON@enronXgate, ""john.j.boudreaux@andersen.com"" \n<'john.j.boudreaux@andersen.com'>@SMTP@enronXgate, \n""kate.e.agnew@us.andersen.com"" \n<'kate.e.agnew@us.andersen.com'>@SMTP@enronXgate, Nicole \nMendez/ENRON@enronXgate \nSubject: Gas Trading Audit Notification\n\n\nThe Gas Trading Audit is scheduled to begin May 11, 2001 and is expected to \ntake approximately two to three months to complete. The audit will be \nexecuted by Arthur Andersen and led by Kate Agnew.\n\nWe will schedule an opening meeting with you the week of May 7, 2001 to \ndiscuss audit scope and any concerns or suggestions you may have.\n\nThe objective of the audit will be to determine whether procedures and \ncontrols exercised over activities are adequate and operating effectively. \n\nResults of the audit will be discussed with you and other appropriate \npersonnel both during the audit and at the audit closing meeting. \n\nWe will endeavor to conduct this audit in a manner that will minimize \ninterruption of your normal business activities and we look forward to \nworking with you and your personnel. Please call me at x5-4554 if have any \nquestions. \n\n\nMechelle Atwood\nDirector, Enron Assurance Services\nEnron Wholesale Services\n713.345.4554; Location: EB2383\nMechelle.Atwood@enron.com\n\n",upcoming audit,0.405648
3,"FYI...........\n---------------------- Forwarded by Jeffrey C Gossett/HOU/ECT on 05/07/2001 \n07:49 AM ---------------------------\nFrom: Mechelle Atwood/ENRON@enronXgate on 05/04/2001 03:24 PM\nTo: Robert L Hall/ENRON@enronXgate, Jeffrey C Gossett/HOU/ECT@ECT, Bryce \nBaxter/HOU/ECT@ECT\ncc: Wes Colwell/ENRON@enronXgate, ""john.j.boudreaux@andersen.com"" \n<'john.j.boudreaux@andersen.com'>@SMTP@enronXgate, \n""kate.e.agnew@us.andersen.com"" \n<'kate.e.agnew@us.andersen.com'>@SMTP@enronXgate, Nicole \nMendez/ENRON@enronXgate \nSubject: Gas Trading Audit Notification\n\n\nThe Gas Trading Audit is scheduled to begin May 11, 2001 and is expected to \ntake approximately two to

# BM25 Search
BM25 algorithm has been around since the 1970s. It is fast and provides different type of searh results from the semantic-type search done above.
It's a bag-of-words type algorithm that looks at query terms within a search document, regardless of their proximity in that document.
More info on https://en.wikipedia.org/wiki/Okapi_BM25

Note: relevance scoring is on a different scale with BM25 than USE.


In [ ]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi

In [ ]:
import sys
sys.path.append ('/content/drive/My Drive/Colab Notebooks/utils/')
from text_preprocessing import *

In [ ]:
#do text preprocessing
params = [nltk_process_text, regex, replace_invalid_chars, remove_blanks]
emails_df['processed_text'] = text_prep(emails_df['content'], params = params)

  0%|          | 12/100000 [00:00<13:57, 119.45it/s]

applying: nltk_process_text


 11%|█         | 10785/100000 [00:00<00:00, 107848.82it/s]

applying: regex


  3%|▎         | 3382/100000 [00:00<00:02, 33807.46it/s]

applying: replace_invalid_chars


  0%|          | 57/100000 [00:00<03:04, 540.83it/s]

applying: remove_blanks


100%|██████████| 100000/100000 [03:43<00:00, 447.48it/s]


In [ ]:
def BM25_search(query):
  query = text_prep(pd.Series(query))
  tokenized_query = query[0].split(" ")
  scores = bm25.get_scores(tokenized_query)
  #look at only the searches with higher relevance scores
  top_n = np.argsort(scores)[::-1][:10]
  result_df = emails_df[['To', 'From','content']].iloc[top_n]
  if len(result_df) > 0:
    relevance_scores = scores[top_n]
    result_df['Relevance Scores'] = relevance_scores
  return result_df

In [ ]:
emails_df[['content', 'processed_text']].head(5)

content  \
186822                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
corpus = emails_df['processed_text'].copy()

In [ ]:
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
#query = input("Enter your search:")
query = "accounting practices"
BM25_search(query)

100%|██████████| 1/1 [00:00<00:00, 332.96it/s]

applying: nltk_process_text
applying: regex
applying: remove_blanks


,To,From,content,Relevance Scores
270526,None,(judy.e.collazo@ssmb.com),"Power & Natural Gas\nMark-to-Market Brouhaha; Buy Energy Marketers\n\n* Recent press has questioned accounting practices of energy\nmarketers. Concerns center on use of ""mark-to-market"" accounting\n* Stocks down 8.7% since Monday\n* View concerns as unfounded\n* Reiterate 1H (Buy, High Risk) ratings on Enron (ENE), Dynegy (DYN)\nand 2M (Outperform, Medium Risk) on Duke Energy (DUK)\n\n <<trc71789.pdf>>\n\nRaymond C. Niles\nPower/Natural Gas Research\nSalomon Smith Barney\n(212) 816-2807\nray.niles@ssmb.com\n\ns\n\n\n\n\n - trc71789.pdf",13.264762
27408,(bob.hall@enron.com),(sally.beck@enron.com),"Here is another one...\n---------------------- Forwarded by Sally Beck/HOU/ECT on 04/11/2001 11:13 AM \n---------------------------\n\n\nMspartalis@aol.com on 04/11/2001 10:51:33 AM\nTo: sally.beck@enron.com\ncc: \nSubject: Accounting Director positions: Robert (Bob) Mickits; Big 5 Energy \nservices\n\n\nSally, \n\nThis is the second of the two emails I am sending you. ? \n\nPlease review the resume of Robert (Bob) Mickits. ?He is currently a \nconsultant with a Big 5 firm, loaded with GAS Energy experience. ?His \nqualifications are: \n* ????CPA; BBA - Accounting \n* ????E&Y \n* ????16 years progressive Accounting/Finance experience and responsibilities \n* ????Developed a business process risk model identifying process \ncharacteristics, business risks and associated best practices for the energy \ntrading and marketing industry. \n* ????Experienced and responsible for front, middle and back office \naccounting, risk management for energy trading and marketing firms. \n* ????Directed all accounting policies & procedures, commodity and \nderivative \naccounting, reviewed and provided analysis on 133 accounting issues. \n* ????Provided clients with infrastructure review and audits for global \ntrading operations. \n* ????Excellent communication skills. \n\nThanks, \nMike \nE. Michael Spartalis, CPA, CPC \nML&R Personnel Solutions \n281-782-3411 (Cell) ?? \nmspartalis@aol.com (temp)\n - Mickits_Robert_0401_rev.doc\n",12.843750
270805,(jpickard@softrax.com),(rosalee.fleming@enron.com),"Julie Pickard <JPickard@softrax.com> on 10/24/2000 05:16:08 PM\nTo: \ncc: SOFTRAX Mkting <Marketing@softrax.com>, Scott Schults \n<SSchults@Softrax.com>, Wes Reuning <WReuning@Softrax.com>, Greta Krupetsky \n<GKrupetsky@softrax.com>, ""Robert O'Connor"" <ROConnor@Softrax.com>, ""Sue \nO'Leary"" <SOLeary@Softrax.com> \nSubject: Revenue Recognition Seminar and Cocktail Reception Invitation fro m \nSOFTRAX\n\n\n If you wish to be removed from this mailing list - please reply and\ntype the word ""remove"" in the subject line.\n\n SOFTRAX Corporation is pleased to invite you to a Financial\nRoundtable discussion on Revenue Recognition. This seminar is free of\ncharge.\n\n Seats are limited, so if you are interested, please contact Julie\nPickard (jpickard@softrax.com) to indicate that you will be attending no\nlater than Monday November 6, 2000. We look forward to seeing you! Please\ninclude your full name, company name, title, email address and phone number.\n\n\n Issues in Accounting for Software and Internet Activities\n Sponsored by: SOFTRAX Corporation\n\n\n Presenter: Ashwinpaul C. Sondhi, PhD\n When: Wednesday, November 15, 2000; 3:00\nPM-5:30 PM\n Cocktail Reception\nto follow 5:30 PM - 7:00 PM\n Where: Omni Austin Hotel, 700 San\nJacinto at 8th Street, Austin, TX 78701. 512.476.3700\n Seminar is held in\nthe Congress Room\n Reception is held in\nthe Chambers Room\n\n Who Should Attend: Controllers, CFO's, CPA's, VC's, and those\ncharged with any aspect of financial reporting for companies\n\n RSVP: Jpickard@softrax.com (by Monday\nNovember 6, 2000)\n\n\n In this discussion you will learn the latest about:\n\n* EITF 00-3: Application of AICPA SOP 97-2 to Arrangements that\nInclude the Right to Use Software stored on Another Entity's Hardware.\n* EITF 99-17: Accounting for Advert

#Summary
Each type of search presented gives different kind of relevance to a given search and is closely related to things like document lenght, text preprocessing etc. 
Future work:
1. optimize code, especially USE-related search
2. normalize search relevance score and create an algorithm to combine searches from different models